<a href="https://colab.research.google.com/github/zackives/upenn-cis-2450/blob/main/8_Module_2_Part_IV_Parallelism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture Module 2 Part 4: Parallelism

## LinkedIn Social Analysis

We now consider extensions of processing to multi-core and multi-machine models.  This includes:

* Polars, a multi-core Pandas-style library that is seeing wide adoption due to its performance (and flexible programming model)
* Sharded execution, a technique that is the basis of Spark's execution model.


In [20]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

File ‘linkedin_anon.jsonl’ already there; not retrieving.



## PennGrader Setup

In [21]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [22]:
!pip3 install penngrader-client

In [23]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [24]:
%set_env HW_ID=cis2450_fall24_HW9

env: HW_ID=cis2450_fall24_HW9


In [25]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


# Loading our Data

Now that we've seen how to do fairly complex queries over data in relations, we'll "pop back" to our big data example, which is the LinkedIn dataset.  Recall that we had a segment of the LinkedIn input file in our previous examples earlier in this module.

In [26]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# Time conversions
import time

In [27]:
'''
Simple code to pull out data from JSON and load into DuckDB.
'''
linked_in = open('linkedin_anon.jsonl')
people_df = pd.read_json('linkedin_anon.jsonl', lines=True)

In [28]:
people_df

,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,tart-acorn,"{'family_name': 'Hannay', 'given_name': 'Passe...","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,https://www.linkedin.com/in/tart-acorn,"[{'major': 'Biophysics', 'end': '2009', 'name'...",None,0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",None,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology",None
3,objective-riesling,"{'family_name': 'Carnegie', 'given_name': 'Pas...",San Francisco Bay Area,None,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,https://www.linkedin.com/in/objective-riesling,None,"{'affilition': ['Big Data, Low Latency', 'Expe...",5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",None,"[{'from': '<Employee Benefits, Administration ...",None,None
4,generative-amberjack,"{'family_name': 'Duncan', 'given_name': 'Merri...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",https://www.linkedin.com/in/generative-amberjack,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,None,None,None,"Literature, Philosophy, Music",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,glowing-flush,"{'family_name': 'Kincaid', 'given_name': 'Pass...",Greater Chicago Area,"[Spanish-speaking, Cicerone-Certified Beer Ser...",Marketing and Advertising,Sales and marketing professional specializing ...,https://www.linkedin.com/in/glowing-flush,"[{'start': '2007', 'major': 'PR and Advertisin...",None,5.0,"[{'org': 'Louis Glunz Beer Inc.', 'title': 'On...",None,"[{'from': 'Peet's Coffee and Tea', 'to': 'Expl...","craft beer industry, coffee industry, running,...",None
49996,grouchy-flight,"{'family_name': 'Ogilvy', 'given_name': 'Bulli...",Greater Atlanta Area,None,Financial Services,Accomplished business development manager expe...,https://www.linkedin.com/in/grouchy-flight,"[{'major': 'Marketing Focus', 'end': '2008', '...","{'member': 'Sigma Chi Fraternity', 'affilition...",14.0,"[{'org': 'Georgia-Pacific LLC', 'title': 'Acco...",None,"[{'from': 'Bayer Advanced', 'to': 'BBDO', 'tit...",None,[National Deans List]
49997,dense-bell,"{'family_name': 'Macdougall', 'given_name': 'B...","Calgary, Canada Area","[Project Management, Electrical Engineering, M...",Design,Brad Gibson is a recognized expert in power qu...,https://www.linkedin.com/in/dense-bell,[{'major': 'Engineering Physics (Solid State E...,"{'member': 'IEEE, APEGGA, APEGBC, PEO, APEGS, ...",42.0,"[{'org': 'DIALOG', 'desc': 'Electrical Enginee...","Data center design, high reliability power, po...","[{'from': 'Current Thinking Inc.', 'to': 'The ...",None,None
49998,brave-hoops,"{'family_name': 'Forsyth', 'given_name': 'Cadb...",San Francisco Bay Area,"[Corporate Social Responsibility, Public Polic...",Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,"[{'major': 'Law', 'end':

In [29]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)

# Take the lists, drop any blank strings
specialties_df = people_df[['_id','specilities']].explode('specilities')
specialties_df.dropna(inplace=True)
interests_df = people_df[['_id','interests']].explode('interests')
interests_df.dropna(inplace=True)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

education_df = get_nested_list_dict(people_df[['_id','education']], 'education')
experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')
skills_df = get_nested_list(people_df[['_id','skills']], 'skills')
honors_df = get_nested_list(people_df[['_id','honors']], 'honors')
events_df = get_nested_list_dict(people_df[['_id','events']], 'events')

groups_df = get_nested_dict(people_df[['_id','group']], 'group')

people_only_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests']).merge(names_df,on='_id')

# Multi-Core Execution of Dataframes

## Polars

Polars is a "drop-in replacement" for Pandas that supports more advanced processing, including multicore processing.

Its syntax is not exactly the same as Pandas', but close.  (In fact, it has some similarities to Apache Spark DataFrames, which we'll be seeing shortly.)

### Polars DataFrames

To take advantage of Polars you want to convert your dataframes from Pandas to Polars. (If you have base files, you can also use Polars' read_csv etc as you would from Pandas.)

In [30]:
import polars as pl

# Convert all Pandas dataframes to Polars Dataframes
people_only_pdf = pl.from_pandas(people_only_df)
education_pdf = pl.from_pandas(education_df)
experience_pdf = pl.from_pandas(experience_df)
skills_pdf = pl.from_pandas(skills_df)
groups_pdf = pl.from_pandas(groups_df)
names_pdf = pl.from_pandas(names_df)

## PL.Col

In Polars, when you are referring to a column in a DataFrame expression, this is represented by a *column object*.  For instance, you can express predicates on the values in a column when filtering, or you can request that certain columns be projected.

You can, e.g., talk about `pl.col('_id')`.  SQL-style, you can also refer to `pl.col('*')` if you are asking for all columns.

## Selection and Projection in Polars

Rather than using Pandas' `x[x[condition]]` notation, Polars has a *filter* function.   To project, you use a function called `select` (this is a bit confusing, but think SQL SELECT, not relational algebra select).

In [31]:
people_only_pdf.\
  select(pl.col('_id','family_name','given_name')).\
  filter(pl.col('family_name') <= 'Smith')

_id,family_name,given_name
str,str,str
"""moist-vodka""","""Post""","""Belvedere"""
"""tart-acorn""","""Hannay""","""Passepartout"""
"""objective-riesling""","""Carnegie""","""Passepartout"""
"""generative-amberjack""","""Duncan""","""Merriman"""
"""chalky-tenement""","""Graham""","""Leporello"""
…,…,…
"""glowing-flush""","""Kincaid""","""Passepartout"""
"""grouchy-flight""","""Ogilvy""","""Bullimore"""
"""dense-bell""","""Macdougall""","""Barrymore"""


## Joins

In many settings, multi-way joins are the most expensive query operations -- partly because they "blow up the data". You saw this with our discussions about join ordering.

We should be able to see the benefits of Polars multicore processing with some join expressions. Note that we still need to keep everything in memory for Pandas/Polars, and in fact there are simple join queries (say, all people joined with experiences and education) that will cause us to run out of memory.

Let's try a simpler query that joins people with some of their other info.

### Pandas Version

In [32]:
%%time
people_only_df.merge(skills_df, on='_id').merge(education_df, on='_id').merge(groups_df, on='_id')

CPU times: user 1.06 s, sys: 116 ms, total: 1.18 s
Wall time: 1.18 s


,_id,locality,industry,summary,url,interval,family_name,given_name,skills,start,major,end,name,desc,degree,affilition,member
0,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,2008,Economics,2008,Columbia University - Columbia Business School,"Coursework ""Principals of Economics"" ECON1105\...",NaN,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
1,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,2007,NaN,2007,Columbia University - Columbia Business School,NaN,NaN,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
2,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1996,Cancer genomics,2001,Göteborgs universitet,"Thesis: ""The role of p53 in tumor progression ...",Ph.D.,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
3,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1994,"Biology, Medicine;German Language",1995,Universität Regensburg,NaN,"Cancer Research, Coursework","[Big Data, Low Latency, Experts Answer's, Link...",NaN
4,adagio-catalyst,"Antwerp Area, Belgium",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,20.0,Watt,Brunton,Strategic Planning,1989,Biology,1994,Göteborgs universitet,,Master,"[Big Data, Low Latency, Experts Answer's, Link...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368157,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1978,International Relations,1981,"University of California, Davis",,BA,NaN,NaN
368158,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,NaN,NaN,NaN,Great Zimbabwe University,,NaN,NaN,NaN
368159,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1981,Law,1984,"University of California, Hastings College of ...",,Juris Doctor,NaN,NaN
368160,brave-hoops,San Francisco Bay Area,Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,26.0,Forsyth,Cadbury,Web Design,1978,International Relations,1981,"University of California, Davis",,BA,NaN,NaN


### Polars Version

Notice this finishes in roughly half the time as Pandas!

In [33]:
%%time
print(people_only_pdf.join(skills_pdf, on='_id').join(education_pdf, on='_id').join(groups_pdf, on='_id'))

shape: (368_162, 17)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬────────┐
│ _id        ┆ locality   ┆ industry  ┆ summary   ┆ … ┆ desc      ┆ degree    ┆ affilitio ┆ member │
│ ---        ┆ ---        ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ n         ┆ ---    │
│ str        ┆ str        ┆ str       ┆ str       ┆   ┆ str       ┆ str       ┆ ---       ┆ str    │
│            ┆            ┆           ┆           ┆   ┆           ┆           ┆ list[str] ┆        │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪════════╡
│ adagio-cat ┆ Antwerp    ┆ Pharmaceu ┆ Ph.D.     ┆ … ┆ Coursewor ┆ null      ┆ ["Big     ┆ null   │
│ alyst      ┆ Area,      ┆ ticals    ┆ scientist ┆   ┆ k "Princi ┆           ┆ Data, Low ┆        │
│            ┆ Belgium    ┆           ┆ with back ┆   ┆ pals of   ┆           ┆ Latency", ┆        │
│            ┆            ┆           ┆ groun…    ┆   ┆ Econ…     ┆   

## Laziness

In [34]:
%%time
result_pdf = people_only_pdf.join(skills_pdf, on='_id').\
  join(education_pdf, on='_id').join(groups_pdf, on='_id')
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf))

10712
CPU times: user 264 ms, sys: 44.5 ms, total: 308 ms
Wall time: 486 ms


In [35]:
%%time
result_pdf = people_only_pdf.lazy().join(skills_pdf.lazy(), on='_id').join(education_pdf.lazy(), on='_id').join(groups_pdf.lazy(), on='_id').lazy()
only_sf_pdf = result_pdf.filter(pl.col('locality') == 'San Francisco Bay Area')
print (len(only_sf_pdf.collect()))

10712
CPU times: user 26.2 ms, sys: 5.88 ms, total: 32.1 ms
Wall time: 60 ms


In [36]:
results = only_sf_pdf.explain()

for line in results.split('\n'):
  print(line)

INNER JOIN:
LEFT PLAN ON: [col("_id")]
  INNER JOIN:
  LEFT PLAN ON: [col("_id")]
    INNER JOIN:
    LEFT PLAN ON: [col("_id")]
      DF ["_id", "locality", "industry", "summary"]; PROJECT */8 COLUMNS; SELECTION: [(col("locality")) == (String(San Francisco Bay Area))]
    RIGHT PLAN ON: [col("_id")]
      DF ["_id", "skills"]; PROJECT */2 COLUMNS; SELECTION: None
    END INNER JOIN
  RIGHT PLAN ON: [col("_id")]
    DF ["_id", "start", "major", "end"]; PROJECT */7 COLUMNS; SELECTION: None
  END INNER JOIN
RIGHT PLAN ON: [col("_id")]
  DF ["_id", "affilition", "member"]; PROJECT */3 COLUMNS; SELECTION: None
END INNER JOIN


# Understanding Sharded Execution

Let's now look (in simulation) at how sharding works!  This is the basis of cluster-based processing.

## A Data Graph in Relations

To do this we'll adapt our LinkedIn data to a *graph* scenario, with people and companies.

In [38]:
persons_df = people_only_df[['_id','family_name','given_name']]

persons_df

,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
3,objective-riesling,Carnegie,Passepartout
4,generative-amberjack,Duncan,Merriman
...,...,...,...
51817,glowing-flush,Kincaid,Passepartout
51818,grouchy-flight,Ogilvy,Bullimore
51819,dense-bell,Macdougall,Barrymore
51820,brave-hoops,Forsyth,Cadbury


In [40]:
experience_df

,_id,org,title,end,start,desc
0,adagio-catalyst,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...
1,adagio-catalyst,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,NaN,September 2008,Single Cell Gene expression.
2,adagio-catalyst,Columbia University,Associate Research Scientist,NaN,August 2006,Work on peptide to restore wt p53 function in ...
3,adagio-catalyst,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,NaN,January 2003,Molecular profiling of colorectal cancer.
4,adagio-catalyst,Sahlgrenska University Hospital,Research Scientist,NaN,November 2001,Cancer Research at Dept of Surgery.Molecular p...
...,...,...,...,...,...,...
1761792,fixed-partition,Four Seasons Hotels and Resorts,Communications Assistant,NaN,1995,NaN
1761793,fixed-partition,M Booth & Associates,"Senior Vice President, Director",Present,March 2002,"Director, Consumer Tech & Travel"
1761794,fixed-partition,Hill & Knowlton,Account Supervisor,NaN,September 1999,NaN
1761795,fixed-partition,Richmond Public Relations,Account Executive,NaN,1996,NaN


Companies are also nodes.  We'll auto-assign IDs for these.

In [42]:
companies_df = experience_df[['org']].drop_duplicates()
companies_df['company_id'] = companies_df.index
companies_df

,org,company_id
0,Johnson and Johnson,0
1,Albert Einstein Medical Center,1
2,Columbia University,2
3,Memorial Sloan Kettering Cancer Center,3
4,Sahlgrenska University Hospital,4
...,...,...
1761646,President's Task Force on Health Care Reform,1761646
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647
1761648,Congresswoman Cardiss Collins,1761648
1761781,M Booth & Associates,1761781


Finally, here are some edges.  We preserve the relationships between people and their experiences with organizations.

In [47]:
worked_for = experience_df[['_id', 'org', 'title']].copy().merge(companies_df.set_index('org'), on='org').drop_duplicates()
worked_for = worked_for.drop(columns=['org'])
worked_for

,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
5,paper-scarfing,GOLD Associate (Global Operations Leadership D...,0
6,paper-scarfing,Summer Intern,0
17,sharp-deck,Professional Sales Representative,0
21,stern-bus,Senior Consultant,0
...,...,...,...
1761753,brave-hoops,Task Force Member,1761646
1761765,brave-hoops,Counsel: U.S. House of Representatives,1761647
1761777,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761789,fixed-partition,"Senior Vice President, Director",1761781


### Data

In [49]:
print('persons_df:')
display(persons_df)
print('companies_df:')
display(companies_df)
print('worked_df:')
display(worked_for)

persons_df:


,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
3,objective-riesling,Carnegie,Passepartout
4,generative-amberjack,Duncan,Merriman
...,...,...,...
51817,glowing-flush,Kincaid,Passepartout
51818,grouchy-flight,Ogilvy,Bullimore
51819,dense-bell,Macdougall,Barrymore
51820,brave-hoops,Forsyth,Cadbury


companies_df:


,org,company_id
0,Johnson and Johnson,0
1,Albert Einstein Medical Center,1
2,Columbia University,2
3,Memorial Sloan Kettering Cancer Center,3
4,Sahlgrenska University Hospital,4
...,...,...
1761646,President's Task Force on Health Care Reform,1761646
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647
1761648,Congresswoman Cardiss Collins,1761648
1761781,M Booth & Associates,1761781


worked_df:


,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
5,paper-scarfing,GOLD Associate (Global Operations Leadership D...,0
6,paper-scarfing,Summer Intern,0
17,sharp-deck,Professional Sales Representative,0
21,stern-bus,Senior Consultant,0
...,...,...,...
1761753,brave-hoops,Task Force Member,1761646
1761765,brave-hoops,Counsel: U.S. House of Representatives,1761647
1761777,brave-hoops,Legislative Counsel: U.S. House of Representat...,1761648
1761789,fixed-partition,"Senior Vice President, Director",1761781


### Implementing Sharded Computations - Our Operators

Rather than look at sharding in Spark, we'll instead simulate having 4 machines, each of which has access to a shard of each table.

To illustrate the basic concepts we'll create our own basic implementations of Relational Algebra over sharded data -- as well as our own `repartition` etc.

In [48]:
nodes = 4

def get_hash(x):
  '''
  Hash code for data, used to compute a shard
  '''
  if isinstance(x, int):
    return x
  elif isinstance(x, str):
    return x.__hash__()
  elif pd.isna(x):
    return 'null'.__hash__()
  else:
    raise RuntimeError('Cannot hash {}'.format(type(x)))

def get_shard(sharded_df: list[pd.DataFrame], index: int):
  '''
  Retrieves the sub-dataframe corresponding to a shard
  '''
  return sharded_df[index]

def repartition(df_or_shards, key: str, shards: int = nodes):
  '''
  Take a dataframe or a list of sharded sub-dataframes.
  Repartition them on the key, based on the number of nodes.
  Return the new list of sub-dataframes.
  '''
  data = df_or_shards
  if isinstance(df_or_shards, list):
    data = pd.concat(df_or_shards)

  resharded_df = []
  for shard in range(0, shards):
    resharded_df.append(data[data[key].apply(lambda x: get_hash(x) % shards == shard)])

  return resharded_df

def rename_shards(sharded_df: list[pd.DataFrame], renamer: dict):
  '''
  Simple relational algebra operator:
  Rename the columns of a list of sharded dataframes
  '''
  renamed_df = []
  for shard in range(0, len(sharded_df)):
    renamed_df.append(sharded_df[shard].rename(columns=renamer))

  return renamed_df

def filter_shards(sharded_df: list[pd.DataFrame], filter_fn):
  '''
  Simple relational algebra operator:
  Filter a list of sharded dataframes with a predicate
  '''
  filtered_df = []
  for shard in range(0, len(sharded_df)):
    filtered_df.append(
        sharded_df[shard][
            sharded_df[shard].apply(filter_fn, axis=1)])

  return filtered_df


def join_shards(sharded_df1: list[pd.DataFrame], sharded_df2: list[pd.DataFrame],
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join two lists of sharded dataframes
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2[shard], left_on=left_key, right_on=right_key))

  return ret_result

def join_shard_with_broadcast(sharded_df1: list[pd.DataFrame], sharded_df2: pd.DataFrame,
                left_key, right_key):
  '''
  Simple relational algebra operator:
  Join a list of sharded dataframes with a single dataframe
  that conceptually gets "broadcast" to all of the shards
  '''
  ret_result = []

  for shard in range(0, len(sharded_df1)):
    ret_result.append(sharded_df1[shard].merge(sharded_df2, left_on=left_key, right_on=right_key))

  return ret_result

## An Example Query

Suppose we simply want to ask this query:

`(persons_df) -[worked_for]--> (companies_df)`

## Shard the Relations!

We'll create versions of this:
* `sharded_persons` (by _id)
* `sharded_companies` (by company)
* Two shards of `worked_for`:
  - `sharded_by_user`
  - `sharded_by_company`

In [70]:
# Generally, for a table like persons, we would
# shard by default on the key (_id).  We can
# see each shard here.

sharded_persons = repartition(persons_df, '_id')

for p in sharded_persons:
  display(p)

,_id,family_name,given_name
0,moist-vodka,Post,Belvedere
8,arctic-investor,Rollo,Merriman
21,impulsive-rook,Cumming,Leporello
30,honest-loop,Nesbitt,Passepartout
37,all-architecture,Mackay,Bertuccio
...,...,...,...
51790,light-foundry,Drummond,Jenkins
51801,calm-block,Macnab,Figaro
51806,espressivo-media,Dundas,Brunton
51815,resultant-coach,Kincaid,Bertuccio


,_id,family_name,given_name
3,objective-riesling,Carnegie,Passepartout
5,salty-section,Watt,Figaro
10,cold-reveal,Kincaid,Barrymore
13,forgiving-desert,Haig,Belvedere
14,future-limburger,Howard,Poole
...,...,...,...
51791,edible-neck,Henderson,Barkley
51796,happy-circuit,Brown,Simonides
51798,glossy-rower,Cunningham,Cadbury
51800,great-twig,Boyle,Cadbury


,_id,family_name,given_name
7,sensitive-estuary,Macneil,Merriman
9,rich-laser,Spalding,Figaro
11,afraid-collision,Hunter,Merriman
12,oily-sport,Burnett,Merriman
23,cheerful-mackerel,Hamilton,Barkley
...,...,...,...
51808,direct-scenario,Boyd,Brunton
51810,resolving-image,Agnew,Jenkins
51811,legato-rook,Keen,Belvedere
51812,flat-leading,Lewis,Cadbury


,_id,family_name,given_name
1,adagio-catalyst,Watt,Brunton
2,tart-acorn,Hannay,Passepartout
4,generative-amberjack,Duncan,Merriman
6,chalky-tenement,Graham,Leporello
17,cheerful-broker,Barclay,Cadbury
...,...,...,...
51813,joint-atoll,Macalister,Cadbury
51814,natural-customer,Charteris,Passepartout
51816,tense-spline,Cunningham,Merriman
51818,grouchy-flight,Ogilvy,Bullimore


In [71]:
# Similarly with companies... we shard by company_id
sharded_companies = repartition(companies_df, 'company_id')

for c in sharded_companies:
  display(c)

,org,company_id
0,Johnson and Johnson,0
4,Sahlgrenska University Hospital,4
32,<Software Consulting Company>,32
40,<Employment Agency>,40
44,<Biotech>,44
...,...,...
1761492,Goose Island Beer Company,1761492
1761576,WestWayne,1761576
1761624,The EA Group,1761624
1761644,CA Manufacturing Technology Center,1761644


,org,company_id
1,Albert Einstein Medical Center,1
25,UCSF,25
29,<Online Recruiting Company>,29
37,<Psychology Organization>,37
41,<Biometrics Hardware Company>,41
...,...,...
1761621,DIALOG,1761621
1761637,The Bipartisan Bridge,1761637
1761641,Nielsen Media's Independent Task Force on Tele...,1761641
1761645,California State Controller’s Office,1761645


,org,company_id
2,Columbia University,2
26,Wyss Institute for Biologically Inspired Engin...,26
30,<Medical Testing Company>,30
34,<Temperature Sensor Manufacturer>,34
42,<Product Manufacturing Company>,42
...,...,...
1761574,Celulosa Arauco,1761574
1761578,Bayer Advanced,1761578
1761638,International Commission on Workforce Development,1761638
1761642,"State of California Business, Transportation, ...",1761642


,org,company_id
3,Memorial Sloan Kettering Cancer Center,3
31,000Montgomery.Com,31
35,<Advertising Company>,35
39,<National Recruitment Network>,39
43,<Emergency Services>,43
...,...,...
1761575,"House Parts, Inc.",1761575
1761623,Current Thinking Inc.,1761623
1761639,"United Nations, Global Alliance on ICT (inform...",1761639
1761647,"Energy and Commerce Subcommittee on Commerce, ...",1761647


Edges are inherently trickier to shard: we can shard by the *source* or by the *destination* but can't simultaneously do so for a single table, by both.  We can, of course, create two dataframes - one sharded by each.

In [53]:
# Edge sharded by user ID
sharded_by_user = repartition(worked_for, '_id')

for edge in sharded_by_user:
  display(edge)

,_id,title,company_id
17,sharp-deck,Professional Sales Representative,0
21,stern-bus,Senior Consultant,0
68,matte-canon,Teacher's Assistant - Unix Administration and ...,2
228,sleek-bar,Administrative Employee,2
256,loud-kelp,Researcher,2
...,...,...,...
1761553,glowing-flush,Full-Time Intern,1761492
1761586,glowing-flush,Brewer's Assistant,1761494
1761595,glowing-flush,Barista,1761497
1761604,glowing-flush,Marketing and Admin Intern,1761498


,_id,title,company_id
38,bold-ship,Various: Staff Radiographer / Instructor,1
49,exhaustive-antagonist,Graduate Research Assistant,2
56,shortest-division,Student,2
79,caramelized-bison,Student,2
80,caramelized-bison,Teaching Assistant,2
...,...,...,...
1761154,glossy-rower,Summer Associate,1760868
1761162,glossy-rower,Engineer Coop,1760870
1761170,glossy-rower,President,1760871
1761789,fixed-partition,"Senior Vice President, Director",1761781


,_id,title,company_id
5,paper-scarfing,GOLD Associate (Global Operations Leadership D...,0
6,paper-scarfing,Summer Intern,0
112,chamfered-vertex,Teaching Assistant,2
115,regular-contract,Research Scientist,2
132,exothermic-seat,Senior Interaction Designer,2
...,...,...,...
1761729,brave-hoops,"Vice President, Government Relations",1761644
1761741,brave-hoops,"Deputy Controller, Legislation",1761645
1761753,brave-hoops,Task Force Member,1761646
1761765,brave-hoops,Counsel: U.S. House of Representatives,1761647


,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
33,adagio-catalyst,Associate at Dept of Molecular Genetics,1
44,adagio-catalyst,Associate Research Scientist,2
57,proud-track,Student,2
63,joint-pie,Graduate Research Assistant,2
...,...,...,...
1761643,grouchy-flight,Assistant Media Planner,1761576
1761650,grouchy-flight,Intern,1761578
1761657,dense-bell,Principal,1761621
1761661,dense-bell,Managing Partner,1761623


Here's the other version...  Sharded by the company ID.

In [54]:
sharded_by_companies = repartition(worked_for, 'company_id')

for edge in sharded_by_companies:
  display(edge)

,_id,title,company_id
0,adagio-catalyst,"Senior Scientist, Oncology Biomarkers",0
5,paper-scarfing,GOLD Associate (Global Operations Leadership D...,0
6,paper-scarfing,Summer Intern,0
17,sharp-deck,Professional Sales Representative,0
21,stern-bus,Senior Consultant,0
...,...,...,...
1761553,glowing-flush,Full-Time Intern,1761492
1761643,grouchy-flight,Assistant Media Planner,1761576
1761665,dense-bell,Manager of Engineering,1761624
1761729,brave-hoops,"Vice President, Government Relations",1761644


,_id,title,company_id
33,adagio-catalyst,Associate at Dept of Molecular Genetics,1
38,bold-ship,Various: Staff Radiographer / Instructor,1
360,tart-acorn,Assistant Professor,25
362,sleek-interval,Research Assistant,25
366,amber-drum,Research Assistant - Department of Psychiatry,25
...,...,...,...
1761657,dense-bell,Principal,1761621
1761669,brave-hoops,Executive Director / Founder,1761637
1761705,brave-hoops,Task Force Member,1761641
1761741,brave-hoops,"Deputy Controller, Legislation",1761645


,_id,title,company_id
44,adagio-catalyst,Associate Research Scientist,2
49,exhaustive-antagonist,Graduate Research Assistant,2
56,shortest-division,Student,2
57,proud-track,Student,2
63,joint-pie,Graduate Research Assistant,2
...,...,...,...
1761629,grouchy-flight,Sales Manager,1761574
1761650,grouchy-flight,Intern,1761578
1761681,brave-hoops,Chief Executive Officer,1761638
1761717,brave-hoops,Deputy Secretary for Information Technology,1761642


,_id,title,company_id
350,adagio-catalyst,Post Doctoral Research Fellow,3
452,objective-riesling,Owner,31
626,objective-riesling,ASP.NET / WSDL / JavaScript,35
627,objective-riesling,ASP.NET / C# / SQL 2005,35
628,objective-riesling,C# / MS Vista / Ant Profiler,35
...,...,...,...
1761636,grouchy-flight,Sales Associate,1761575
1761661,dense-bell,Managing Partner,1761623
1761693,brave-hoops,Strategy Council Member,1761639
1761765,brave-hoops,Counsel: U.S. House of Representatives,1761647


## Let's Do Some Computation!

### Single Join: People -> works_for

To do this, we need to be careful to join only between tables that are sharded by the join key (_id).

In [55]:
# sharded_persons is sharded by _id
# sharded_by_user is worked_for sharded by _id
for table in join_shards(sharded_persons, sharded_by_user, '_id', '_id'):
  display(table)

# Show the final results of the join
display(pd.concat(join_shards(sharded_persons, sharded_by_user, '_id', '_id')))

,_id,family_name,given_name,title,company_id
0,honest-loop,Nesbitt,Passepartout,Vice President / General Manager,1789
1,honest-loop,Nesbitt,Passepartout,Account Management Head,1790
2,honest-loop,Nesbitt,Passepartout,Business Unit Manager CNS & Pain,1790
3,honest-loop,Nesbitt,Passepartout,Business Unit Manager CV & Metabolic,1790
4,honest-loop,Nesbitt,Passepartout,Group Product Manager Men's Health & NSAIDs,1790
...,...,...,...,...,...
63842,glowing-flush,Kincaid,Passepartout,Full-Time Intern,1761492
63843,glowing-flush,Kincaid,Passepartout,Brewer's Assistant,1761494
63844,glowing-flush,Kincaid,Passepartout,Barista,1761497
63845,glowing-flush,Kincaid,Passepartout,Marketing and Admin Intern,1761498


,_id,family_name,given_name,title,company_id
0,objective-riesling,Carnegie,Passepartout,Ning,29
1,objective-riesling,Carnegie,Passepartout,.Net 4.0 / C# / VS 2010 / Silverlight 4.0 / Bl...,30
2,objective-riesling,Carnegie,Passepartout,Owner,31
3,objective-riesling,Carnegie,Passepartout,ASP.NET / C# / SQL 2005 / XML / XSD,32
4,objective-riesling,Carnegie,Passepartout,PHP / MySQL,32
...,...,...,...,...,...
66314,glossy-rower,Cunningham,Cadbury,President,1760871
66315,fixed-partition,Donald,Alfred,Account Supervisor,155055
66316,fixed-partition,Donald,Alfred,Communications Assistant,197439
66317,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781


,_id,family_name,given_name,title,company_id
0,sensitive-estuary,Macneil,Merriman,Sales Manager - Security Team,979
1,sensitive-estuary,Macneil,Merriman,Information Security Consultant,979
2,sensitive-estuary,Macneil,Merriman,IT Security Sales Consultant,981
3,sensitive-estuary,Macneil,Merriman,Sales Manager,982
4,sensitive-estuary,Macneil,Merriman,Corporate Account Manager,983
...,...,...,...,...,...
62554,brave-hoops,Forsyth,Cadbury,"Vice President, Government Relations",1761644
62555,brave-hoops,Forsyth,Cadbury,"Deputy Controller, Legislation",1761645
62556,brave-hoops,Forsyth,Cadbury,Task Force Member,1761646
62557,brave-hoops,Forsyth,Cadbury,Counsel: U.S. House of Representatives,1761647


,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
64093,grouchy-flight,Ogilvy,Bullimore,Intern,1761578
64094,dense-bell,Macdougall,Barrymore,Project Manager,937293
64095,dense-bell,Macdougall,Barrymore,Principal,1761621
64096,dense-bell,Macdougall,Barrymore,Managing Partner,1761623


,_id,family_name,given_name,title,company_id
0,honest-loop,Nesbitt,Passepartout,Vice President / General Manager,1789
1,honest-loop,Nesbitt,Passepartout,Account Management Head,1790
2,honest-loop,Nesbitt,Passepartout,Business Unit Manager CNS & Pain,1790
3,honest-loop,Nesbitt,Passepartout,Business Unit Manager CV & Metabolic,1790
4,honest-loop,Nesbitt,Passepartout,Group Product Manager Men's Health & NSAIDs,1790
...,...,...,...,...,...
64093,grouchy-flight,Ogilvy,Bullimore,Intern,1761578
64094,dense-bell,Macdougall,Barrymore,Project Manager,937293
64095,dense-bell,Macdougall,Barrymore,Principal,1761621
64096,dense-bell,Macdougall,Barrymore,Managing Partner,1761623


So, 256823 rows!

We can prove the above is correct by looking at the standard, unsharded result.

In [58]:
# Traditional Pandas query, check the count and results
persons_df.merge(worked_for, left_on='_id', right_on='_id')

,_id,family_name,given_name,title,company_id
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0
1,adagio-catalyst,Watt,Brunton,Associate at Dept of Molecular Genetics,1
2,adagio-catalyst,Watt,Brunton,Associate Research Scientist,2
3,adagio-catalyst,Watt,Brunton,Post Doctoral Research Fellow,3
4,adagio-catalyst,Watt,Brunton,Research Scientist,4
...,...,...,...,...,...
256818,brave-hoops,Forsyth,Cadbury,Legislative Counsel: U.S. House of Representat...,1761648
256819,fixed-partition,Donald,Alfred,Account Supervisor,155055
256820,fixed-partition,Donald,Alfred,Communications Assistant,197439
256821,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781


### What If We Don't Watch Our Sharding?

What if we don't align the shard columns and join keys? Let's see...

In [56]:
# sharded_persons is sharded by _id
# sharded_by_companies is worked_for sharded by the company

# Output the result of the join without proper sharding
pd.concat(join_shards(sharded_persons, sharded_by_companies, '_id', '_id'))

,_id,family_name,given_name,title,company_id
0,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796
1,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796
2,all-architecture,Mackay,Bertuccio,Sr. Research scientist,1984
3,thin-level,Macfarlane,Leporello,Caixa On-Line,2632
4,thin-level,Macfarlane,Leporello,Caixa Executiva,2632
...,...,...,...,...,...
15501,natural-customer,Charteris,Passepartout,District Team Leader - Alabama,47699
15502,natural-customer,Charteris,Passepartout,Store Team Leader,47699
15503,natural-customer,Charteris,Passepartout,District Team Leader - Oklahoma,1761443
15504,grouchy-flight,Ogilvy,Bullimore,Sales Associate,1761575


What happens if we *broadcast* one of the relations?

We would generally only do this with a small table.

In [59]:
pd.concat(join_shard_with_broadcast(sharded_persons, worked_for, '_id', '_id'))

,_id,family_name,given_name,title,company_id
0,honest-loop,Nesbitt,Passepartout,Vice President / General Manager,1789
1,honest-loop,Nesbitt,Passepartout,Account Management Head,1790
2,honest-loop,Nesbitt,Passepartout,Business Unit Manager CNS & Pain,1790
3,honest-loop,Nesbitt,Passepartout,Business Unit Manager CV & Metabolic,1790
4,honest-loop,Nesbitt,Passepartout,Group Product Manager Men's Health & NSAIDs,1790
...,...,...,...,...,...
64093,grouchy-flight,Ogilvy,Bullimore,Intern,1761578
64094,dense-bell,Macdougall,Barrymore,Project Manager,937293
64095,dense-bell,Macdougall,Barrymore,Principal,1761621
64096,dense-bell,Macdougall,Barrymore,Managing Partner,1761623


## Now Let's Do a Full Connection

OK, we saw one join.  If we want to traverse our network from people to companies, we need two joins.

Here's the Pandas query we want to do in a sharded model...

In [60]:
persons_df.merge(worked_for, left_on='_id', right_on='_id').merge(companies_df, left_on='company_id', right_on='company_id')

,_id,family_name,given_name,title,company_id,org
0,adagio-catalyst,Watt,Brunton,"Senior Scientist, Oncology Biomarkers",0,Johnson and Johnson
1,paper-scarfing,Montgomery,Cadbury,GOLD Associate (Global Operations Leadership D...,0,Johnson and Johnson
2,paper-scarfing,Montgomery,Cadbury,Summer Intern,0,Johnson and Johnson
3,sharp-deck,Maclaren,Barrymore,Professional Sales Representative,0,Johnson and Johnson
4,stern-bus,Buchanan,Barkley,Senior Consultant,0,Johnson and Johnson
...,...,...,...,...,...,...
256818,brave-hoops,Forsyth,Cadbury,Task Force Member,1761646,President's Task Force on Health Care Reform
256819,brave-hoops,Forsyth,Cadbury,Counsel: U.S. House of Representatives,1761647,"Energy and Commerce Subcommittee on Commerce, ..."
256820,brave-hoops,Forsyth,Cadbury,Legislative Counsel: U.S. House of Representat...,1761648,Congresswoman Cardiss Collins
256821,fixed-partition,Donald,Alfred,"Senior Vice President, Director",1761781,M Booth & Associates


To join with `companies_df` we need to repartition our intermediate result on the company.

In [62]:
# This is the join from above
persons_work_for = join_shards(sharded_persons, sharded_by_user, '_id', '_id')

# persons_work_for is sharded by _id
# sharded_companies is sharded by company
# to join them, we need to find a common sharding key -- the company

# SO: we repartition persons_work_for by company
persons_companies = join_shards(repartition(persons_work_for, 'company_id'), sharded_companies, 'company_id', 'company_id')

pd.concat(
persons_companies
)

,_id,family_name,given_name,title,company_id,org
0,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796,Boehringer Ingelheim Pharmaceuticals
1,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals
2,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals
3,all-architecture,Mackay,Bertuccio,Sr. Research scientist,1984,Merck Development centre
4,thin-level,Macfarlane,Leporello,Caixa On-Line,2632,Banco Itaú
...,...,...,...,...,...,...
62829,joint-atoll,Macalister,Cadbury,CEO and Founder,1761247,BTS Automated Systems
62830,joint-atoll,Macalister,Cadbury,Program Manager,1761255,Clifford Chance Rogers and Wells
62831,natural-customer,Charteris,Passepartout,District Team Leader - Oklahoma,1761443,Target Stores
62832,grouchy-flight,Ogilvy,Bullimore,Sales Associate,1761575,"House Parts, Inc."


Again, this matches our Pandas query.

## Two Hops

Can we find employees who have a common employer?

Let's divide it into steps.

First, let's do a building block, namely the set of people, companies they work for, and the interconnecting `works` relationships:

```
people --> works_for --> company <-- works_for
[---- (persons_companies) -----]
```

In [65]:
# sharded_by_companies is the works_for edge, sharded by company
second_works_for = rename_shards(sharded_by_companies, {'_id': '_id2', 'role': 'role2'})

# We are currently sharded on company
person_works_company_works = \
    filter_shards(join_shards(persons_companies, second_works_for, 'company_id', 'company_id'),
           lambda row: row['_id'] != row['_id2']
    )
pd.concat(person_works_company_works)

,_id,family_name,given_name,title_x,company_id,org,_id2,title_y
2,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative
5,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative
6,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Product Manager Primary Care
7,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Pharmaceutical Sales Representative
12,thin-level,Macfarlane,Leporello,Caixa On-Line,2632,Banco Itaú,linear-apparition,Systems Analyst
...,...,...,...,...,...,...,...,...
748082,clear-boat,Akins,Cadbury,Account Executive - Renault/Dacia and Stihl/Vi...,1488955,Publicis Conseil,irregular-snapper,Junior Strategic Planner
748858,terminal-space,Mackay,Figaro,Channel Sales Director,1602199,"KnowNow, Inc",watery-scale,"President, CEO"
748859,watery-scale,Chattan,Leporello,"President, CEO",1602199,"KnowNow, Inc",terminal-space,Channel Sales Director
749145,frayed-flow,Brodie,Barrymore,IT Consultant,1653635,Evosoft,intense-convertible,Software Engineer


Now we can join with another company of the

In [66]:
sharded_persons_2 = rename_shards(sharded_persons, {'_id': '_id2', 'given_name': 'given_name2', 'last_name': 'last_name2'})

result = join_shards(repartition(person_works_company_works, '_id2'), sharded_persons_2, '_id2', '_id2')
# Take the above and shard + join with sharded_persons
for table in result:
  display(table)

display(pd.concat(result))

,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Product Manager Primary Care,Nesbitt,Passepartout
1,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Pharmaceutical Sales Representative,Nesbitt,Passepartout
2,adjacent-inference,Murray,Bertuccio,"Group Leader, Global Advanced Technical Services",1789,Mylan Pharmaceuticals,honest-loop,Vice President / General Manager,Nesbitt,Passepartout
3,merry-triplex,Mackay,Cadbury,SAP BI Consultant,1789,Mylan Pharmaceuticals,honest-loop,Vice President / General Manager,Nesbitt,Passepartout
4,fuchsia-quanta,Wood,Simonides,Area Sales Manager,1790,Pfizer,honest-loop,Account Management Head,Nesbitt,Passepartout
...,...,...,...,...,...,...,...,...,...,...
1286248,rust-vault,Maclaren,Barkley,Algemeen directeur,1554779,Noordwest12 bv,animato-bench,Account Director,Spalding,Simonides
1286249,weighted-gain,Beckham,Simonides,Desarrollador,339827,Desarrollador Independiente,pizzicato-deque,Desarrollador,Forrester,Simonides
1286250,salty-classifier,Strange,Passepartout,IT Manager Brazil,1074891,Subsea7,isometric-collision,Reporting Surveyor/ Senior Data Processor,Boyd,Merriman
1286251,average-ton,Carmichael,Bullimore,Human Resources Administrator,65511,Shop At Home Network,weary-sitrep,Website Product Assistant,Ogilvy,Barrymore


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,gentle-sensor,Bruce,Barrymore,Lead Technical Analyst,4976,AMD,chewy-bankruptcy,ASIC Design Intern,Macmillan,Brunton
1,andante-chaise,Finlay,Barrymore,AMD Fellow,4976,AMD,chewy-bankruptcy,ASIC Design Intern,Macmillan,Brunton
2,archaic-sectional,Malcolm,Bullimore,"Senior Product Manager, Tablet & Security Soft...",4976,AMD,chewy-bankruptcy,ASIC Design Intern,Macmillan,Brunton
3,archaic-sectional,Malcolm,Bullimore,Senior Technical Marketing Manager – Multimedi...,4976,AMD,chewy-bankruptcy,ASIC Design Intern,Macmillan,Brunton
4,intense-estuary,Montgomery,Bullimore,"Director, Silicon Engineering",4976,AMD,chewy-bankruptcy,ASIC Design Intern,Macmillan,Brunton
...,...,...,...,...,...,...,...,...,...,...
1340061,small-emporium,Lennox,Jeeves,Manager Projects,249011,Seamless Distribution AB,stone-bronze,VP Sales,Robertson,Brunton
1340062,hot-sardine,Macdougall,Merriman,System Administrator,249011,Seamless Distribution AB,stone-bronze,VP Sales,Robertson,Brunton
1340063,kinetic-canister,Hay,Simonides,Assistant Researcher,1165811,University of Leipzig,corn-ambiance,PhD student,Becker,Merriman
1340064,kinetic-canister,Hay,Simonides,Assistant Researcher,1165811,University of Leipzig,corn-ambiance,Student research assistant,Becker,Merriman


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
1,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
2,azure-globe,Fergusson,Alfred,"Account Executive, Technical Recruiter",526646,Outsource Technical,minimum-screw,"Sr. Director, Business Development - Midwest R...",Drummond,Cadbury
3,azure-globe,Fergusson,Alfred,"Account Executive, Technical Recruiter",526646,Outsource Technical,minimum-screw,"Sr. Account Executive, Client Relationship Man...",Drummond,Cadbury
4,gentle-sensor,Bruce,Barrymore,Lead Technical Analyst,4976,AMD,isochoric-treble,Principal Member of Technical Staff (PMTS),Montgomery,Poole
...,...,...,...,...,...,...,...,...,...,...
1279389,seasoned-cilantro,Keen,Simonides,Senior Brand Manager,167539,PT. Unilever Indonesia,kind-content,Marketing Trainee,Stuart,Merriman
1279390,seasoned-cilantro,Keen,Simonides,Consumer Activation Manager,167539,PT. Unilever Indonesia,kind-content,Brand Manager,Stuart,Merriman
1279391,seasoned-cilantro,Keen,Simonides,Consumer Activation Manager,167539,PT. Unilever Indonesia,kind-content,Assistant Brand Manager,Stuart,Merriman
1279392,seasoned-cilantro,Keen,Simonides,Consumer Activation Manager,167539,PT. Unilever Indonesia,kind-content,Marketing Trainee,Stuart,Merriman


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,thin-level,Macfarlane,Leporello,Caixa On-Line,2632,Banco Itaú,linear-apparition,Systems Analyst,Haldane,Passepartout
1,thin-level,Macfarlane,Leporello,Caixa Executiva,2632,Banco Itaú,linear-apparition,Systems Analyst,Haldane,Passepartout
2,stable-loop,Macdonnell,Jeeves,Investment Advisor,2632,Banco Itaú,linear-apparition,Systems Analyst,Haldane,Passepartout
3,flat-bunk,Napier,Leporello,Teacher,106368,Universidade Anhembi Morumbi,linear-apparition,Systems Analyst,Haldane,Passepartout
4,extended-convolution,Crichton,Barrymore,Analista Programador Sênior,106369,Senac,linear-apparition,Internet Technologies Teacher,Haldane,Passepartout
...,...,...,...,...,...,...,...,...,...,...
1297206,intricate-cottage,Irvine,Passepartout,commissielid,1414487,VVD,crazy-reduction,lid,Mackay,Leporello
1297207,icy-classic,Kincaid,Alfred,eLearning Consultant,1452943,Istanbul Bilgi University,closed-tesla,Part-time Instructor,Chattan,Leporello
1297208,grouchy-hertz,Nesbitt,Merriman,Part-time research assistant,1452943,Istanbul Bilgi University,closed-tesla,Part-time Instructor,Chattan,Leporello
1297209,irregular-snapper,Dundas,Alfred,Junior Strategic Planner,1488955,Publicis Conseil,clear-boat,Account Executive - Renault/Dacia and Stihl/Vi...,Akins,Cadbury


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Product Manager Primary Care,Nesbitt,Passepartout
1,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Pharmaceutical Sales Representative,Nesbitt,Passepartout
2,adjacent-inference,Murray,Bertuccio,"Group Leader, Global Advanced Technical Services",1789,Mylan Pharmaceuticals,honest-loop,Vice President / General Manager,Nesbitt,Passepartout
3,merry-triplex,Mackay,Cadbury,SAP BI Consultant,1789,Mylan Pharmaceuticals,honest-loop,Vice President / General Manager,Nesbitt,Passepartout
4,fuchsia-quanta,Wood,Simonides,Area Sales Manager,1790,Pfizer,honest-loop,Account Management Head,Nesbitt,Passepartout
...,...,...,...,...,...,...,...,...,...,...
1297206,intricate-cottage,Irvine,Passepartout,commissielid,1414487,VVD,crazy-reduction,lid,Mackay,Leporello
1297207,icy-classic,Kincaid,Alfred,eLearning Consultant,1452943,Istanbul Bilgi University,closed-tesla,Part-time Instructor,Chattan,Leporello
1297208,grouchy-hertz,Nesbitt,Merriman,Part-time research assistant,1452943,Istanbul Bilgi University,closed-tesla,Part-time Instructor,Chattan,Leporello
1297209,irregular-snapper,Dundas,Alfred,Junior Strategic Planner,1488955,Publicis Conseil,clear-boat,Account Executive - Renault/Dacia and Stihl/Vi...,Akins,Cadbury


In [73]:
persons_2 = persons_df.rename(columns={'_id': '_id2', 'given_name': 'given_name2', 'last_name': 'last_name2'})

result = join_shard_with_broadcast(person_works_company_works, persons_2, '_id2', '_id2')

for table in result:
  display(table)

display(pd.concat(result))

,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
1,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
2,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Product Manager Primary Care,Nesbitt,Passepartout
3,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Pharmaceutical Sales Representative,Nesbitt,Passepartout
4,thin-level,Macfarlane,Leporello,Caixa On-Line,2632,Banco Itaú,linear-apparition,Systems Analyst,Haldane,Passepartout
...,...,...,...,...,...,...,...,...,...,...
1771373,simmered-barbette,Chattan,Cadbury,Technical Inside Sales Coordinator,1601040,Tyco Thermal Controls,formal-camp,Web Project Manager Europe,Macnab,Jeeves
1771374,simmered-barbette,Chattan,Cadbury,Technical Inside Sales Coordinator,1601040,Tyco Thermal Controls,formal-camp,MarCom/Web Specialist,Macnab,Jeeves
1771375,tough-pole,Hunter,Barkley,Business Specialist II,1686884,Raymond James Financial,abstract-bucket,Leadership Development Consultant,Stuart,Figaro
1771376,bold-molecule,Douglas,Jeeves,Software Developer [Hired],1705760,Tribal Internet Solutions,violent-crate,Webdesigner / front-end developer,Macduff,Belvedere


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Vice President / General Manager,1789,Mylan Pharmaceuticals,merry-triplex,SAP BI Consultant,Mackay,Cadbury
1,adjacent-inference,Murray,Bertuccio,"Group Leader, Global Advanced Technical Services",1789,Mylan Pharmaceuticals,merry-triplex,SAP BI Consultant,Mackay,Cadbury
2,sleek-allegory,Armstrong,Passepartout,Revenue Accountant/Contract,468397,AES,merry-triplex,SAP BW Consultant,Mackay,Cadbury
3,angry-cabin,Guthrie,Brunton,Administrateur Système,221193,La Poste,merry-triplex,Chef de projet BI,Mackay,Cadbury
4,bronze-gauge,Dunbar,Leporello,Expert Informatique Distribuée,221193,La Poste,merry-triplex,Chef de projet BI,Mackay,Cadbury
...,...,...,...,...,...,...,...,...,...,...
836879,diagonal-electron,Macnab,Barrymore,Kortlægningsassistent og studentermedhjælp,1647149,GEUS,tart-mass,Project Leader,Nicolson,Merriman
836880,tart-mass,Nicolson,Merriman,Project Leader,1647149,GEUS,diagonal-electron,Kortlægningsassistent og studentermedhjælp,Macnab,Barrymore
836881,ash-vector,Cumming,Passepartout,Sales Associate,1648901,Michaels Arts and Crafts,closed-consulate,Front End Supervisor | Senior Framer,Becker,Brunton
836882,closed-consulate,Becker,Brunton,Front End Supervisor | Senior Framer,1648901,Michaels Arts and Crafts,ash-vector,Sales Associate,Cumming,Passepartout


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Account Management Head,1790,Pfizer,fuchsia-quanta,Area Sales Manager,Wood,Simonides
1,honest-loop,Nesbitt,Passepartout,Business Unit Manager CNS & Pain,1790,Pfizer,fuchsia-quanta,Area Sales Manager,Wood,Simonides
2,honest-loop,Nesbitt,Passepartout,Business Unit Manager CV & Metabolic,1790,Pfizer,fuchsia-quanta,Area Sales Manager,Wood,Simonides
3,honest-loop,Nesbitt,Passepartout,Group Product Manager Men's Health & NSAIDs,1790,Pfizer,fuchsia-quanta,Area Sales Manager,Wood,Simonides
4,obsolete-beam,Spalding,Bullimore,"Senior Advisor, Research Business Technology",1790,Pfizer,fuchsia-quanta,Area Sales Manager,Wood,Simonides
...,...,...,...,...,...,...,...,...,...,...
1856857,clever-buyer,Hunter,Simonides,"Vice President, National Accounts",1565498,PMI Mortgage Insurance Co.,burning-feist,Contract Underwriter,Robertson,Leporello
1856858,clever-buyer,Hunter,Simonides,National Accounts Vice President,1565498,PMI Mortgage Insurance Co.,burning-feist,Contract Underwriter,Robertson,Leporello
1856859,binary-balcony,Murdoch,Barkley,Duty Manager,1604214,The Park Hotels,shiny-peak,Human Resources,Chisholm,Cadbury
1856860,shiny-peak,Chisholm,Cadbury,Human Resources,1604214,The Park Hotels,binary-balcony,Duty Manager,Murdoch,Barkley


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Product Team Leader,1795,Eli Lilly,ripe-beef,Executive Director & Chief Operating Officer -...,Mackay,Barkley
1,honest-loop,Nesbitt,Passepartout,Product Team Leader,1795,Eli Lilly,ripe-beef,Managing Director - Lilly University,Mackay,Barkley
2,honest-loop,Nesbitt,Passepartout,Product Team Leader,1795,Eli Lilly,ripe-beef,Director - Knowledge Management,Mackay,Barkley
3,honest-loop,Nesbitt,Passepartout,Product Team Leader,1795,Eli Lilly,ripe-beef,Manager - Discovery Strategy/Productivity,Mackay,Barkley
4,honest-loop,Nesbitt,Passepartout,Product Team Leader,1795,Eli Lilly,ripe-beef,Productivity Projects Associate,Mackay,Barkley
...,...,...,...,...,...,...,...,...,...,...
737795,irregular-snapper,Dundas,Alfred,Junior Strategic Planner,1488955,Publicis Conseil,clear-boat,Account Executive - Renault/Dacia and Stihl/Vi...,Akins,Cadbury
737796,clear-boat,Akins,Cadbury,Account Executive - Renault/Dacia and Stihl/Vi...,1488955,Publicis Conseil,irregular-snapper,Junior Strategic Planner,Dundas,Alfred
737797,terminal-space,Mackay,Figaro,Channel Sales Director,1602199,"KnowNow, Inc",watery-scale,"President, CEO",Chattan,Leporello
737798,frayed-flow,Brodie,Barrymore,IT Consultant,1653635,Evosoft,intense-convertible,Software Engineer,Finlay,Cadbury


,_id,family_name_x,given_name,title_x,company_id,org,_id2,title_y,family_name_y,given_name2
0,honest-loop,Nesbitt,Passepartout,Product Manager Primary Care,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
1,honest-loop,Nesbitt,Passepartout,Pharmaceutical Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,minimum-screw,Professional Sales Representative,Drummond,Cadbury
2,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Product Manager Primary Care,Nesbitt,Passepartout
3,minimum-screw,Drummond,Cadbury,Professional Sales Representative,1796,Boehringer Ingelheim Pharmaceuticals,honest-loop,Pharmaceutical Sales Representative,Nesbitt,Passepartout
4,thin-level,Macfarlane,Leporello,Caixa On-Line,2632,Banco Itaú,linear-apparition,Systems Analyst,Haldane,Passepartout
...,...,...,...,...,...,...,...,...,...,...
737795,irregular-snapper,Dundas,Alfred,Junior Strategic Planner,1488955,Publicis Conseil,clear-boat,Account Executive - Renault/Dacia and Stihl/Vi...,Akins,Cadbury
737796,clear-boat,Akins,Cadbury,Account Executive - Renault/Dacia and Stihl/Vi...,1488955,Publicis Conseil,irregular-snapper,Junior Strategic Planner,Dundas,Alfred
737797,terminal-space,Mackay,Figaro,Channel Sales Director,1602199,"KnowNow, Inc",watery-scale,"President, CEO",Chattan,Leporello
737798,frayed-flow,Brodie,Barrymore,IT Consultant,1653635,Evosoft,intense-convertible,Software Engineer,Finlay,Cadbury


## Exercise

Let's do this via a broadcast join.

In [ ]:
# TODO
result = join_shard_with_broadcast(# TODO
)

display(pd.concat(result))

In [ ]:
grader.grade('sharded_net_result', len(pd.concat(result)))